In [1]:
from numba import njit
from random import randint as rndint
from random import random as rnd
import numpy as np
from math import sqrt

ModuleNotFoundError: No module named 'numba'

In [2]:
@njit
# checks if position is in board
def isInBoard(position, dim):
    x,y = position
    return (0<=x<dim and 0<=y<dim)

@njit
# finds number of queens hits
def totHits(positions):
    hits = 0
    dim = len(positions)
    for i in range(dim):

        # moves up right
        (x,y) = (i+1, positions[i]+1)
        while isInBoard((x,y), dim):
            if positions[x] == y:
                hits+=1
            x+=1
            y+=1
        # moves down right
        (x,y) = (i+1, positions[i]-1)
        while isInBoard((x,y), dim):
            if positions[x] == y:
                hits+=1
            x+=1
            y-=1
        # moves up left
        (x,y) = (i-1, positions[i]+1)
        while isInBoard((x,y), dim):
            if positions[x] == y:
                hits+=1
            x-=1
            y+=1
        (x,y) = (i-1, positions[i]-1)
        # moves down left
        while isInBoard((x,y), dim):
            if positions[x] == y:
                hits+=1
            x-=1
            y-=1
    return hits

@njit
def evaluate(positions):
    return -totHits(positions)/2

In [3]:
@njit
def crossover(elem1, elem2, crossRatio):
    size = len(elem1)
    if rnd() >= crossRatio:
        return elem1.copy(), elem2.copy()
    crossPoint1 = rndint(0,size-1)
    # crossPoint2 = rndint(0,size-1)
    crossPoint2 = size-1
    while crossPoint1 == crossPoint2:
        crossPoint2 = rndint(0,size-1)
    # if crossPoint1 > crossPoint2:
    #     crossPoint1, crossPoint2 = crossPoint2, crossPoint1
    child1 = np.full(size, -1, np.int_)
    child2 = np.full(size, -1, np.int_)
    for i in range(crossPoint1, crossPoint2+1):
        child1[i] = elem1[i]
        child2[i] = elem2[i]
    free1 = crossPoint2+1
    free2 = crossPoint2+1
    for i in range(crossPoint2+1, crossPoint2+size+1):
        j = i % size
        if elem1[j] not in child2:
            free2 = free2 % size
            child2[free2] = elem1[j]
            free2 += 1
        if elem2[j] not in child1:
            free1 = free1 % size
            child1[free1] = elem2[j]
            free1 += 1
    return child1, child2

In [4]:
@njit
def tweak(positions):
    l = len(positions)
    x = rndint(0,l-1)
    y = rndint(0,l-1)
    while(x==y):
        y = rndint(0,l-1)
    positions[x], positions[y] = positions[y], positions[x]

@njit
#random starting population
def populationInitialize(populationSize, size):
    population = np.empty((populationSize, size), np.int_)
    for i in range(populationSize):
        element = np.arange(size, dtype=np.int_)
        for _ in range(10 * size):
            tweak(element)
        population[i] = element
    return population

@njit
def mutation(elem, mutationRatio):
    if rnd() < mutationRatio:
        tweak(elem)

In [5]:
@njit
def populationScores(flattenedPopulation, populationSize, size):
    population = flattenedPopulation.reshape((populationSize, size))
    scores = np.empty(populationSize, np.int_)
    for i in range(populationSize):
        scores[i] = evaluate(population[i])
    return scores

In [6]:
# rank based selection
@njit
def index(v):
    dis = 1 + (8*v)
    return int((-1+(sqrt(dis)))/2)

@njit
def rankSelection(flattenPopulation, populationSize, size, scores, totRanks):
    population = flattenPopulation.reshape((populationSize, size))
    pop_scores = sorted(zip(population, scores), key=lambda x: x[1])
    selected = np.empty((len(population), size), dtype=np.int_)
    for i in range(len(population)):
        elemi = index(rndint(0, totRanks-1))
        selected[i] = pop_scores[elemi][0]
    return selected

In [11]:
@njit
def geneticAlgorithm(size, populationSize, iterations, crossRatio, mutationRatio):
    population = populationInitialize(populationSize, size)
    scores = populationScores(population.flatten(), populationSize, size)
    bestScore = scores.max()
    indexBestElement = np.where(scores == bestScore)[0][0]
    bestElement = population[indexBestElement]
    totRanks = int(populationSize * (populationSize + 1) / 2)
    for _ in range(iterations):
        selected = rankSelection(population.flatten(), populationSize, size, scores, totRanks)
        childs = np.empty((populationSize, size), dtype = np.int_)
        for i in range(0, populationSize, 2):
            child1, child2 = crossover(selected[i], selected[i+1], crossRatio)
            mutation(child1, mutationRatio)
            mutation(child2, mutationRatio)
            childs[i] = child1
            childs[i+1] = child2
        population = childs
        scores = populationScores(population.flatten(), populationSize, size)
        bestChildScore = scores.max()
        if bestChildScore > bestScore:
            indexBestElement = np.where(scores == bestChildScore)[0][0]
            bestElement = population[indexBestElement]
            bestScore = bestChildScore
        if bestScore == 0:
            break
    print(bestElement, bestScore)

In [12]:
from multiprocessing import Process
def geneticParallel(cores, size, populationSize, iterations, crossRatio, mutationRatio):
    processes = [None] * cores
    for i in range(cores):
        processes[i] = Process(target = geneticAlgorithm,
                            args = (size, populationSize, iterations, crossRatio,
                                    mutationRatio))
        processes[i].start()
    for i in range(cores):
        processes[i].join()

In [13]:
if __name__ == "__main__":
    from time import perf_counter
    size = 20
    iterations = 10000
    populationSize = 14
    crossRatio =0.90
    mutationRatio = 0.1
    fails = 0
    its = 30
    cores = 2

    start = perf_counter()
    for _ in range(its):
        geneticParallel(cores, size, populationSize, iterations, crossRatio, mutationRatio)
    end = perf_counter()

    print("average time:", (end-start)/its)
    print("fail ratio:", int(100*fails/its), "%")

[11  3 10 17  9  5 15  1  6 16 14  8  4  2 13 18  7  0 19 12] 0
[15 17  6 10 16 14  1  5  2  8 18  0 12  3  7 11 13 19  9  4] 0
[ 8 16 13  6  2 15 18  9  3  5 11 17  0 12  4  7 10 14  1 19] 0
[14 17  7  4 16  0 10  5 11  1 12  2  9 19  3  8 15 13 18  6] 0
[ 2 14 12 15  1  4  7  3 17 13 18  5 19 11  9  6 16 10  8  0] 0
[ 7  5 14  9  0  8 17 15  6 19 11 16 12  4  1  3 10  2 13 18] 0
[ 5  1 18  6 11 16  0  4 17 19  9 12 14  3 10  7  2 15 13  8] 0
[13  9  0  8 19 16  2 11  1 10  4 18 14  3  6 12 15 17  7  5] 0


KeyboardInterrupt: 